In [1]:
import logging                                                                                                                                                                                                                                                                                                          
import matplotlib.pyplot as plt                                                                                                                                                                                                                                                                                         
import numpy as np                                                                                                                                                                                                                                                                                                      
import pandas as pd                                                                                                                                                                                                                                                                                                     
import pickle as pkl                                                                                                                                                                                                                                                                                                    
from sklearn.model_selection import train_test_split                                                                                                                                                                                                                                                                    
from sklearn.preprocessing import StandardScaler                                                                                                                                                                                                                                                                        
import torch                                                                                                                                                                                                                                                                                                            
import torch.nn as nn                                                                                                                                                                                                                                                                                                   
import torch.utils.data as utils                                                                                                                                                                                                                                                                                        
                                                                                                                                                                                                                                                                                                                        
import sys                                                                                                                                                                                                                                                                                                              
import os                                                                                                                                                                                                                                                                                                               
sys.path.append("./..")                                                                                                                                                                                                                                                     
                                                                                                                                                                                                                                                                                                                        
from common.argparser import eigan_argparse   
from common.text_processing import text_preprocessing

from common.utility import log_shapes, log_time, torch_device, time_stp, load_processed_data, logger, sep, weights_init                                                                                                                                                                                                                                                            
from common.torchsummary import summary                                                                                                                                                                                                                                                                                 
                                                                                                                                                                                                                                                                                                                        
from models.eigan import GeneratorFCN, DiscriminatorFCN     

In [2]:
device='gpu'
expt='mimic'

In [ ]:
device = torch_device(device=device)
device

In [ ]:
df_data = pd.read_csv('../checkpoints/mimic/patients_join_admissions.csv')
df_data.fillna('na', axis=1, inplace=True)
df_data.head(3)

In [ ]:
len_prev = len(df_data)
df_data.drop([
    'row_id', 
    'row_id.1', 
    'subject_id', 
    'subject_id.1',
    'dod',
    'dod_hosp',
    'dod_ssn',
    'hadm_id',
], axis=1, inplace=True)
len_curr = len(df_data)
print("{} rows with null values dropped".format(len_prev-len_curr))
print("{} rows of data available".format(len_curr))

In [ ]:
df_data.head(3)

In [ ]:
for col in [
    'dob',
    'admittime',
    'dischtime',
    'deathtime',
    'edregtime',
    'edouttime',
]:
    df_data[col][df_data[col]=='na'] = pd.Timestamp('18000101')
    df_data[col+"_year"] = pd.DatetimeIndex(df_data[col]).year
    df_data[col+"_month"] = pd.DatetimeIndex(df_data[col]).month
    df_data[col+"_day"] = pd.DatetimeIndex(df_data[col]).day
    df_data.drop(col, axis=1, inplace=True)

In [ ]:
for col in [
    'admission_type', 
    'admission_location', 
    'discharge_location', 
    'insurance',
    'religion',
    'marital_status',
    'ethnicity',
    'diagnosis',
]:
    df_data[col] = list(map(text_preprocessing, df_data[col].tolist()))
df_data.head(3)